In [2]:
import yaml
import requests
import pandas as pd

In [3]:
API_KEY = yaml.full_load(open('creds.yaml', 'r'))['maps_api']

In [4]:
dd = requests.get('https://gbfs.baywheels.com/gbfs/en/station_information.json')
stations = pd.DataFrame(dd.json()['data']['stations'])

In [5]:
stations.loc[stations['name'].str.contains('Caltrain')]

,station_type,legacy_id,electric_bike_surcharge_waiver,eightd_station_services,short_name,external_id,lon,name,station_id,has_kiosk,capacity,region_id,rental_uris,rental_methods,eightd_has_key_dispenser,lat
128,classic,465,False,[],SF-J29-3,a7da1857-747a-4507-9d8f-2a9f95f0babf,-122.394495,San Francisco Caltrain Station (King St at 4th...,a7da1857-747a-4507-9d8f-2a9f95f0babf,True,18,3,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,"[KEY, CREDITCARD]",False,37.776311
237,classic,130,False,[],SF-P30,e7691d93-eed6-4388-b681-00ae0b455d23,-122.391885,22nd St Caltrain Station,e7691d93-eed6-4388-b681-00ae0b455d23,True,35,3,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,"[KEY, CREDITCARD]",False,37.757704
364,classic,30,False,[],SF-J29,ea11aa7f-c070-46a4-a074-70604340efa2,-122.395282,San Francisco Caltrain (Townsend St at 4th St),ea11aa7f-c070-46a4-a074-70604340efa2,True,19,3,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,"[KEY, CREDITCARD]",False,37.776598


In [6]:
lat1, lon1 = stations.loc[stations['name'] == 'Commercial St at Montgomery St'][['lat', 'lon']].iloc[0]

In [7]:
lat2, lon2 = stations.loc[stations['name'] == 'San Francisco Caltrain Station (King St at 4th St)'][['lat', 'lon']].iloc[0]

In [8]:
req = requests.get("https://maps.googleapis.com/maps/api/directions/json?avoid=highways&destination={},{}&mode=bicycling&origin={},{}&key={}".format(lat1, lon1, lat2, lon2, API_KEY))

In [9]:
res = req.json()

In [10]:
res

{'geocoded_waypoints': [{'geocoder_status': 'OK',
   'place_id': 'ChIJiZsGldZ_j4ARSAUO2zvLtxM',
   'types': ['establishment', 'point_of_interest']},
  {'geocoder_status': 'OK',
   'place_id': 'ChIJAyk-84qAhYARoYww6NIvE6A',
   'types': ['establishment', 'point_of_interest']}],
 'routes': [{'bounds': {'northeast': {'lat': 37.7968933, 'lng': -122.3875728},
    'southwest': {'lat': 37.7762883, 'lng': -122.4028589}},
   'copyrights': 'Map data ©2022 Google',
   'legs': [{'distance': {'text': '2.4 mi', 'value': 3849},
     'duration': {'text': '15 mins', 'value': 920},
     'end_address': 'Commercial St at Montgomery St, San Francisco, CA 94111, USA',
     'end_location': {'lat': 37.7942628, 'lng': -122.4028589},
     'start_address': 'San Francisco Caltrain Station (King St at 4th St), San Francisco, CA 94107, USA',
     'start_location': {'lat': 37.7762883, 'lng': -122.394524},
     'steps': [{'distance': {'text': '308 ft', 'value': 94},
       'duration': {'text': '1 min', 'value': 17},
 

In [11]:
res.keys()

dict_keys(['geocoded_waypoints', 'routes', 'status'])

In [12]:
import datetime as dt

str(dt.timedelta(seconds=res['routes'][0]['legs'][0]['duration']['value']))

'0:15:20'

In [13]:
def convert_sec_to_time(num_sec):
    return str(dt.timedelta(seconds=res['routes'][0]['legs'][0]['duration']['value']))


def get_travel_time(req):
    return req['routes'][0]['legs'][0]['duration']['value']


def get_stations_coord(name, stations_df):
    if name in stations_df['name'].values:
        return stations_df.loc[stations_df['name'] == name][['lat', 'lon']].iloc[0]
    return None, None

def make_request(station1, station2, stations_df):
    lat1, lon1 = get_stations_coord(station1, stations_df)
    lat2, lon2 = get_stations_coord(station2, stations_df)
    
    req = requests.get("https://maps.googleapis.com/maps/api/directions/json?avoid=highways&destination={},{}&mode=bicycling&origin={},{}&key={}".format(lat1, lon1, lat2, lon2, API_KEY))
    
    return req.json()

In [14]:
lat, lon = get_stations_coord('Commercial St at Montgomery St', stations)

In [15]:
make_request('Commercial St at Montgomery St', 'San Francisco Caltrain Station (King St at 4th St)', stations)

{'geocoded_waypoints': [{'geocoder_status': 'OK',
   'place_id': 'ChIJiZsGldZ_j4ARSAUO2zvLtxM',
   'types': ['establishment', 'point_of_interest']},
  {'geocoder_status': 'OK',
   'place_id': 'ChIJAyk-84qAhYARoYww6NIvE6A',
   'types': ['establishment', 'point_of_interest']}],
 'routes': [{'bounds': {'northeast': {'lat': 37.7968933, 'lng': -122.3875728},
    'southwest': {'lat': 37.7762883, 'lng': -122.4028589}},
   'copyrights': 'Map data ©2022 Google',
   'legs': [{'distance': {'text': '2.4 mi', 'value': 3849},
     'duration': {'text': '15 mins', 'value': 920},
     'end_address': 'Commercial St at Montgomery St, San Francisco, CA 94111, USA',
     'end_location': {'lat': 37.7942628, 'lng': -122.4028589},
     'start_address': 'San Francisco Caltrain Station (King St at 4th St), San Francisco, CA 94107, USA',
     'start_location': {'lat': 37.7762883, 'lng': -122.394524},
     'steps': [{'distance': {'text': '308 ft', 'value': 94},
       'duration': {'text': '1 min', 'value': 17},
 

In [16]:
lat1, lon1 = [3.77860194e+01, -1.22403728e+02]

In [ ]:
def manhattan_distance(a, b):
    return sum(abs(val1-val2) for val1, val2 in zip(a,b))

In [17]:
stations

,station_type,legacy_id,electric_bike_surcharge_waiver,eightd_station_services,short_name,external_id,lon,name,station_id,has_kiosk,capacity,region_id,rental_uris,rental_methods,eightd_has_key_dispenser,lat
0,classic,426,False,[],SJ-M4,68c89d1f-407a-4550-a2b7-ecf0ad7ee422,-121.913497,San Carlos St at Meridian Ave,68c89d1f-407a-4550-a2b7-ecf0ad7ee422,True,19,5,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,"[KEY, CREDITCARD]",False,37.323345
1,classic,404,False,[],SJ-M14,c53990d7-f965-40f4-b305-3435e1c95a71,-121.870678,21st St at Santa Clara St,c53990d7-f965-40f4-b305-3435e1c95a71,True,19,5,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,"[KEY, CREDITCARD]",False,37.345759
2,classic,418,False,[],SJ-L13,30429aed-9a47-4fd9-87fa-0db835aa8265,-121.874385,17th St at Santa Clara St,30429aed-9a47-4fd9-87fa-0db835aa8265,True,19,5,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,"[KEY, CREDITCARD]",False,37.343985
3,classic,422,False,[],SJ-J6,d75591d7-080d-46cb-8ada-0fbe6af676fc,-121.909149,Stockton Ave at Lenzen Ave,d75591d7-080d-46cb-8ada-0fbe6af676fc,True,23,5,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,"[KEY, CREDITCARD]",False,37.336775
4,classic,288,False,[],SJ-H10,0d730ac1-7ce6-45cf-aca6-b412ea46709d,-121.902016,Mission St at 1st St,0d730ac1-7ce6-45cf-aca6-b412ea46709d,True,27,5,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,"[KEY, CREDITCARD]",False,37.350964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,classic,440,False,[],SF-M15,dd34aaa2-dd91-4d83-9f92-e8a2f1cc2f93,-122.449228,Carl St at Cole St,dd34aaa2-dd91-4d83-9f92-e8a2f1cc2f93,True,23,3,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,"[KEY, CREDITCARD]",False,37.765942
490,classic,577,False,[],SF-C18,82ad4cff-001f-49a8-88fa-a5a63d64b9a2,-122.442729,Divisadero St at Lombard St,577,True,22,NaN,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,"[KEY, CREDITCARD]",False,37.799175
491,classic,578,False,[],SF-X16,41b2200f-7915-410b-95c6-084bc4d3dee8,-122.444305,Detroit St at Hearst Ave,578,True,19,NaN,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,"[KEY, CREDITCARD]",False,37.730655
492,classic,579,False,[],SJ-H16,93d95ded-d49e-43bc-b7fe-6ce556569088,-121.871612,Berryessa BART Station (Mabury Rd at Lenfest Rd),579,True,25,5,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,"[KEY, CREDITCARD]",False,37.365088
